# RNN

Recurrent Neural Network (RNN) is a class of neural networks whose connections between
neurons form a directed cycle. Unlike feedforward neural networks, RNN can use its internal
“memory” to process a sequence of inputs, which makes it popular for processing sequential
information. The “memory” means that RNN performs the same task for every element of a
sequence with each output being dependent on all previous computations, which is like
“remembering” information about what has been processed so far.
<img src=rnn1.png>
The left graph is an unfolded network with cycles, while the
right graph is a folded sequence network with three time steps. The length of time steps is
determined by the length of input. For example, if the word sequence to be processed is a sentence
of six words, the RNN would be unfolded into a neural network with six time steps or layers. One
layer corresponds to a word.


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected

In [3]:
n_steps = 28
n_inputs = 28
n_neurons = 128
n_outputs = 10
learning_rate = 0.001

mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = fully_connected(states, n_outputs, activation_fn=None)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [5]:
n_epochs = 5
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test[:1000], y: y_test[:1000]})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.94 Test accuracy: 0.928
1 Train accuracy: 0.926667 Test accuracy: 0.944
2 Train accuracy: 0.98 Test accuracy: 0.95
3 Train accuracy: 0.966667 Test accuracy: 0.96
4 Train accuracy: 0.96 Test accuracy: 0.958
